In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini") 

### Second concept: The Message

In [3]:
from autogen_agentchat.messages import TextMessage

messages = TextMessage(content="I want to play stardew valley", source="user")
messages

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 1, 7, 31, 42, 885621, tzinfo=datetime.timezone.utc), content='I want to play stardew valley', type='TextMessage')

### Third concept: The Agent

In [9]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="game_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for a game. You give short, update, informative, and humorous answers",
    model_client_stream=True
)

### Put it all together with on_messages

In [10]:
from autogen_core import CancellationToken

response = await agent.on_messages([messages], cancellation_token=CancellationToken())
response.chat_message.content

"Great choice! Stardew Valley is like a warm hug from a pixelated farm! Just remember: crops don't water themselves, and those chickens might judge your farming skills. Happy farming! 🌾🐔"

### Let's make a local database of ticket prices

In [33]:
import os 
import sqlite3

# Delete existing database file if it exists
if os.path.exists("games.db"):
    os.remove("games.db")

# Create database and table
conn = sqlite3.connect("games.db")
c = conn.cursor()
c.execute("CREATE TABLE games (name TEXT PRIMARY KEY, price REAL)")
conn.commit()
conn.close()

In [34]:
# Populate database

def save_game_price(name, price):
    conn = sqlite3.connect("games.db")
    c = conn.cursor()
    c.execute("REPLACE INTO games (name, price) VALUES (?, ?)", (name.lower(), price))
    conn.commit()
    conn.close()

# Some Games
save_game_price("Stardew Valley", 150)
save_game_price("PUBG: BATTLEGROUNDS", 0)
save_game_price("METAL GEAR SOLID Δ: SNAKE EATER", 67)
save_game_price("Red Dead Redemption 2", 15)
save_game_price("Indiana Jones and the Great Circle", 67)

In [35]:
# Method to get price for city
def get_game_price(name: str) -> float | None:
    """ Get the game price to buy and play the game"""
    
    conn = sqlite3.connect("games.db")
    c = conn.cursor()
    c.execute("SELECT price FROM games WHERE name = ?", (name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [36]:
get_game_price("Stardew Valley")

150.0

In [37]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_game_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for a game. You give short, informative, humorouse answer, including the price of the game",
    model_client_stream=True,
    tools=[get_game_price],
    reflect_on_tool_use=True # We don't want just return the tool result but able to take continue process after tool has returned
)

In [38]:
response = await smart_agent.on_messages([messages], cancellation_token=CancellationToken())

for inner_message in response.inner_messages:
    print(inner_message.content)

response.chat_message.content

[FunctionCall(id='call_sfsrLsfpwjjc1IadsOe0ZYvU', arguments='{"name":"Stardew Valley"}', name='get_game_price')]
[FunctionExecutionResult(content='150.0', name='get_game_price', call_id='call_sfsrLsfpwjjc1IadsOe0ZYvU', is_error=False)]


"Great choice! Stardew Valley is the perfect escape to a rural paradise where you can farm, mine, and make friends (or date your favorite pixelated characters). It’s currently priced at $150. Just remember to water those crops or they'll turn into sad little brown squares! 🌾😊"